AREEBA NAEEM

MUHAMMAD TAHA MANSOOR

231236

231234

BS-AI-5A

# ASSIGNMENT

adding agents of

Coordinator

ResearchAgent

AnalysisAgent

MemoryAgent

In [37]:

!pip install gradio scikit-learn

import gradio as gr
import datetime
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# ---------------- MEMORY AGENT ---------------- #
class MemoryAgent:
    def __init__(self):
        self.conversations = []
        self.knowledge = []

    def embed(self, text):
        # Simple lightweight embedding (vector simulation)
        return np.array([len(text), sum(ord(c) for c in text) % 1000])

    def store_conversation(self, query, agents):
        self.conversations.append({
            "time": str(datetime.datetime.now()),
            "query": query,
            "agents": agents
        })

    def store_knowledge(self, topic, content, source, confidence):
        self.knowledge.append({
            "topic": topic,
            "content": content,
            "source": source,
            "confidence": confidence,
            "embedding": self.embed(content),
            "time": str(datetime.datetime.now())
        })

    def retrieve(self, query):
        if not self.knowledge:
            return "No prior knowledge available."

        q_emb = self.embed(query)
        scores = []

        for item in self.knowledge:
            sim = cosine_similarity([q_emb], [item["embedding"]])[0][0]
            scores.append((sim, item))

        scores.sort(reverse=True, key=lambda x: x[0])
        return scores[0][1]["content"]

# ---------------- RESEARCH AGENT (ENRICHED) ---------------- #
class ResearchAgent:
    def __init__(self):
        self.db = {
            "neural networks": [
                "Neural networks are inspired by the human brain, consisting of layers of interconnected nodes (neurons).",
                "Main types of neural networks include:",
                "- CNNs (Convolutional Neural Networks) for image processing and computer vision.",
                "- RNNs (Recurrent Neural Networks) for sequence data like text and time series.",
                "- Transformers, which rely on self-attention mechanisms and are state-of-the-art for NLP tasks.",
                "Other types include Autoencoders, GANs (Generative Adversarial Networks), and Spiking Neural Networks.",
                "Neural networks are widely used in AI applications such as classification, prediction, and generative modeling."
            ],
            "transformer architectures": [
                "Transformers use self-attention mechanisms that allow them to process all input tokens in parallel.",
                "They enable models to capture long-range dependencies better than RNNs.",
                "Popular transformer models include BERT, GPT, and T5.",
                "They are widely used in NLP, computer vision, and multimodal tasks."
            ],
            "reinforcement learning": [
                "Reinforcement learning (RL) is a type of machine learning where an agent learns by interacting with an environment.",
                "It uses rewards or penalties to optimize behavior over time.",
                "Common methods include Q-learning, Policy Gradient, DQN, and Actor-Critic algorithms.",
                "Challenges include exploration vs exploitation, sample inefficiency, and reward shaping."
            ]
        }

    def run(self, query):
        q = query.lower()
        for key in self.db:
            if key in q:
                print(f"🔍 ResearchAgent matched topic: {key}")
                return self.db[key]
        return ["No relevant information found. You may try another query."]

# ---------------- ANALYSIS AGENT ---------------- #
class AnalysisAgent:
    def analyze(self, data):
        if "No relevant information" in data:
            return "Analysis could not be performed due to insufficient data."

        result = "Analysis:\n"
        for d in data:
            result += f"- {d}\n"

        result += "\nConclusion: Each approach has trade-offs depending on application."
        return result

# ---------------- COORDINATOR (MANAGER) ---------------- #
class Coordinator:
    def __init__(self):
        self.memory = MemoryAgent()
        self.research = ResearchAgent()
        self.analysis = AnalysisAgent()

    def route(self, query):
        q = query.lower()
        if "what did we discuss" in q or "earlier" in q:
            return "MEMORY"
        if "analyze" in q or "compare" in q:
            return "COMPLEX"
        return "SIMPLE"

    def handle(self, query):
        agents = []
        route = self.route(query)

        # MEMORY QUERY
        if route == "MEMORY":
            agents.append("MemoryAgent")
            response = self.memory.retrieve(query)

        # SIMPLE QUERY
        elif route == "SIMPLE":
            agents.append("ResearchAgent")
            data = self.research.run(query)
            response = "\n".join(data)
            self.memory.store_knowledge(query, response, "ResearchAgent", 0.7)

        # COMPLEX QUERY
        else:
            agents.extend(["ResearchAgent", "AnalysisAgent"])
            data = self.research.run(query)
            response = self.analysis.analyze(data)
            self.memory.store_knowledge(query, response, "AnalysisAgent", 0.85)

        self.memory.store_conversation(query, agents)

        trace = f"\n\n🧠 Agents Used: {', '.join(agents)}"
        return response + trace

# ---------------- GRADIO UI (UPDATED) ---------------- #
coordinator = Coordinator()

def chat_system(user_input, selected_agent):
    # If user selects a specific agent, bypass Coordinator routing
    if selected_agent == "ResearchAgent":
        data = coordinator.research.run(user_input)
        response = "\n".join(data)
        coordinator.memory.store_knowledge(user_input, response, "ResearchAgent", 0.7)
    elif selected_agent == "AnalysisAgent":
        data = coordinator.research.run(user_input)
        response = coordinator.analysis.analyze(data)
        coordinator.memory.store_knowledge(user_input, response, "AnalysisAgent", 0.85)
    elif selected_agent == "MemoryAgent":
        response = coordinator.memory.retrieve(user_input)
    else:  # Coordinator default routing
        response = coordinator.handle(user_input)

    return response

interface = gr.Interface(
    fn=chat_system,
    inputs=[
        gr.Textbox(lines=3, placeholder="Ask your question here..."),
        gr.Dropdown(
            choices=["Coordinator", "ResearchAgent", "AnalysisAgent", "MemoryAgent"],
            value="Coordinator",
            label="Choose Agent (Optional)"
        )
    ],
    outputs=gr.Textbox(lines=15, label="Multi-Agent System Response"),  # Increased size
    title="Multi-Agent Chat System ",
    description="Coordinator dynamically routes queries. Optionally, select a specific agent."
)

interface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b824a63058309d2d4f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
